In [36]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [37]:
#loading the mnist dataset
(mnist_train, mnist_test), mnist_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [38]:
#preprocessing data
def normalize_img(image, label):
    """uint8 -> float32"""
    return tf.cast(image, tf.float32) /255., label

#normalizing features and mapping to float32
mnist_train = mnist_train.map(
    normalize_img,
    num_parallel_calls=tf.data.AUTOTUNE
)

#caching for better performance
mnist_train = mnist_train.cache()

#randomly shuffle the data
mnist_train = mnist_train.shuffle(mnist_info.splits['train'].num_examples)

#batch size of 128
mnist_train = mnist_train.batch(128)

#for better performance
mnist_train = mnist_train.prefetch(tf.data.AUTOTUNE)

In [39]:
#preparing testing
mnist_test = mnist_test.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)

#batch size of 128
mnist_test = mnist_test.batch(128)

#caching for better performance
mnist_test = mnist_test.cache()
mnist_test = mnist_test.prefetch(tf.data.AUTOTUNE)

In [40]:
#our model: softmax classification
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [54]:
#adam
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.Accuracy],
)
model.fit(
    mnist_train,
    epochs=45,
    validation_data=mnist_test,
)

TypeError: 'property' object is not iterable

In [53]:
#adagrad
model.compile(
    optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.001, initial_accumulator_value=0.1, epsilon=1e-08),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.Accuracy()],
)
model.fit(
    mnist_train,
    epochs=1,
    validation_data=mnist_test,
)

ValueError: in user code:

    File "/home/chaudhary/.local/share/virtualenvs/adam_adagrad_nesterovSGD-PHVAGlpI/lib/python3.8/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/home/chaudhary/.local/share/virtualenvs/adam_adagrad_nesterovSGD-PHVAGlpI/lib/python3.8/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/chaudhary/.local/share/virtualenvs/adam_adagrad_nesterovSGD-PHVAGlpI/lib/python3.8/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/home/chaudhary/.local/share/virtualenvs/adam_adagrad_nesterovSGD-PHVAGlpI/lib/python3.8/site-packages/keras/engine/training.py", line 817, in train_step
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "/home/chaudhary/.local/share/virtualenvs/adam_adagrad_nesterovSG